In [1]:
import os
import shutil
import random

source_base_dir = '/kaggle/input/image-pro/Image_Processing'  


base_dir = '/kaggle/working/'


original_dir = os.path.join(source_base_dir, 'Original')
pixelated_dir = os.path.join(source_base_dir, 'Pixelated')


train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')


os.makedirs(os.path.join(train_dir, 'Original'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'Pixelated'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'Original'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'Pixelated'), exist_ok=True)


split_ratio = 0.8

def split_data(original_dir, pixelated_dir, train_dir, test_dir, split_ratio):
    original_files = os.listdir(original_dir)
    pixelated_files = os.listdir(pixelated_dir)

    
    paired_files = [(file, file) for file in original_files if file in pixelated_files]

    random.shuffle(paired_files)
    split_index = int(len(paired_files) * split_ratio)
    train_files = paired_files[:split_index]
    test_files = paired_files[split_index:]

    for original_file, pixelated_file in train_files:
        shutil.copy(os.path.join(original_dir, original_file), os.path.join(train_dir, 'Original', original_file))
        shutil.copy(os.path.join(pixelated_dir, pixelated_file), os.path.join(train_dir, 'Pixelated', pixelated_file))
    
    for original_file, pixelated_file in test_files:
        shutil.copy(os.path.join(original_dir, original_file), os.path.join(test_dir, 'Original', original_file))
        shutil.copy(os.path.join(pixelated_dir, pixelated_file), os.path.join(test_dir, 'Pixelated', pixelated_file))

split_data(original_dir, pixelated_dir, train_dir, test_dir, split_ratio)

In [4]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Load the training and validation datasets
train_dir = '/kaggle/working/train'
test_dir = '/kaggle/working/test'

train_ds = image_dataset_from_directory(train_dir,
                                        image_size=(224, 224),
                                        batch_size=32,
                                        label_mode='binary')

test_ds = image_dataset_from_directory(test_dir,
                                       image_size=(224, 224),
                                       batch_size=32,
                                       label_mode='binary')

# Normalize the images
normalization_layer = tf.keras.layers.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))


2024-06-14 09:42:33.454727: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-14 09:42:33.454861: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-14 09:42:33.604029: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 7884 files belonging to 2 classes.
Found 1972 files belonging to 2 classes.


In [5]:
from tensorflow.keras import layers, models

def build_model():
    model = models.Sequential([
        hub.KerasLayer("https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1", 
                       trainable=False, input_shape=(224, 224, 3)),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

model = build_model()
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              4049564   
                                                                 
 dense (Dense)               (None, 1)                 1281      
                                                                 
Total params: 4050845 (15.45 MB)
Trainable params: 1281 (5.00 KB)
Non-trainable params: 4049564 (15.45 MB)
_________________________________________________________________


In [6]:
history = model.fit(train_ds,
                    validation_data=test_ds,
                    epochs=10)


Epoch 1/10
247/247 [==============================] - 430s 2s/step - loss: 0.4458 - accuracy: 0.8038 - val_loss: 0.3345 - val_accuracy: 0.8707
Epoch 2/10
247/247 [==============================] - 412s 2s/step - loss: 0.2873 - accuracy: 0.8932 - val_loss: 0.2775 - val_accuracy: 0.8900
Epoch 3/10
247/247 [==============================] - 380s 2s/step - loss: 0.2427 - accuracy: 0.9126 - val_loss: 0.2528 - val_accuracy: 0.8971
Epoch 4/10
247/247 [==============================] - 421s 2s/step - loss: 0.2183 - accuracy: 0.9205 - val_loss: 0.2385 - val_accuracy: 0.9026
Epoch 5/10
247/247 [==============================] - 422s 2s/step - loss: 0.2021 - accuracy: 0.9257 - val_loss: 0.2286 - val_accuracy: 0.9067
Epoch 6/10
247/247 [==============================] - 376s 2s/step - loss: 0.1904 - accuracy: 0.9309 - val_loss: 0.2214 - val_accuracy: 0.9097
Epoch 7/10
247/247 [==============================] - 402s 2s/step - loss: 0.1808 - accuracy: 0.9346 - val_loss: 0.2159 - val_accuracy: 0.9138

In [7]:
loss, accuracy = model.evaluate(test_ds)
print(f"Test Accuracy: {accuracy:.2f}")


62/62 [==============================] - 73s 1s/step - loss: 0.2059 - accuracy: 0.9189
Test Accuracy: 0.92


In [8]:
model.save('/kaggle/working/pixelation_detector_model')
